In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn import neighbors, datasets
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

import matplotlib.pyplot as plt

## Read

In [2]:
df = pd.read_csv('./job7_all_clean.csv',encoding='utf-8-sig')

In [3]:
# data clean drop na 
df = df.dropna(thresh=29).reset_index(drop=True) #有內文 電池可有可無
print(df.isnull().sum())
print(len(df["職缺連結"].values))
print(sum(df.isnull().sum()))
print(df.shape)

職缺連結        0
職缺類別        0
職位類別        0
職位          0
大職業編號       0
           ..
金旭飯店管裡系統    0
金蝶          0
鉅茂          0
鼎基-ERP      0
鼎新          0
Length: 693, dtype: int64
57127
238623
(57127, 693)


In [4]:
# 三種版本

# df 有內文 電池可有可無
# df

# 有內文 有電池
df = df.dropna().reset_index(drop=True) 

# 沒內文
# df = df[df.isnull().any(axis=1)].reset_index(drop=True) 

print(df.shape)

(21958, 693)


## df

In [5]:
#check col num
print(df.columns)
print(len(df.columns))

Index(['職缺連結', '職缺類別', '職位類別', '職位', '大職業編號', '中職業編號', '小職業編號', '縣市', '地區',
       '縣市編碼',
       ...
       '英文打字150以上', '英文打字20~50', '英文打字20以下', '英文打字50~75', '英文打字75~100',
       '金旭飯店管裡系統', '金蝶', '鉅茂', '鼎基-ERP', '鼎新'],
      dtype='object', length=693)
693


In [6]:
# print raw data before get dummies
print((df.iloc[:,0:43]).columns)
print((df.iloc[:,193:]).columns)
#48-186(不含) 科系
#193到底 擅長工具

print(set(df["工作經歷"].values))
print(set(df["職位"].values))
print(set(df["需求人數"].values))
print(set(df["學歷要求"].values))
print(set(df["工作經歷"].values))
print(set(df['工作性質'].values))
print(set(df['管理責任'].values))
print(set(df['工作待遇_min'].values))

Index(['職缺連結', '職缺類別', '職位類別', '職位', '大職業編號', '中職業編號', '小職業編號', '縣市', '地區',
       '縣市編碼', '地區編碼', '公司連結', '供需人數', '職缺編號', '公司名稱', '職缺名稱', '更新日期', '工作內容',
       '職務類別', '工作待遇', '工作性質', '上班地點', '管理責任', '上班時段', '需求人數', '工作經歷', '學歷要求',
       '科系要求', '擅長工具', '工作技能', '其他條件', '福利制度', '學歷分佈', '性別分佈', '工作經驗分佈',
       '年齡分佈', '語言能力分佈', '科系分佈', '技能分佈', '證照分佈', '資本額', '員工人數', '公司內文'],
      dtype='object')
Index(['3G', '3ds-Max', '3ds-Max-Design', '6-Sigma', 'A+', 'ABAQUS', 'ADA',
       'ADC', 'ADO', 'ADSL',
       ...
       '英文打字150以上', '英文打字20~50', '英文打字20以下', '英文打字50~75', '英文打字75~100',
       '金旭飯店管裡系統', '金蝶', '鉅茂', '鼎基-ERP', '鼎新'],
      dtype='object', length=500)
{'5-10年', '0-1年', '10年以上', '3-5年', '1-3年'}
{'MES工程師', '系統分析師', '通訊軟體工程師', '其他資訊專業人員', '韌體設計工程師', '電子商務技術主管', '資料庫管理人員', 'BIOS工程師', '資訊設備管制人員', '電玩程式設計師', '演算法開發工程師', '系統維護_操作人員', '軟體專案主管', 'MIS_網管主管', '網路管理工程師', 'Internet程式設計師', '資訊助理人員', '網路安全分析師', 'MIS程式設計師', '軟體設計工程師'}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 18, 20,

In [7]:
# label encoding

edu_dict = {'高中':18,'專科':20, '大學':22, '碩士':25, '博士':30}
df['學歷要求'] = df['學歷要求'].map(edu_dict)
print(set(df['學歷要求'].values))

working_time = {"日班":1,'輪班':2}
df["上班時段_label"] = df['上班時段'].map(working_time)

tw_region_label = {"臺北市":1,"新北市":2,"臺中市":3,"新竹市":4,"桃園市":5,"高雄市":6,"新竹縣":7, '新竹市':8,"臺南市":9,"苗栗縣":10
,"彰化縣":11,"雲林縣":12,"宜蘭縣":13,"嘉義縣":14,"花蓮縣":15,"屏東縣":16,"南投縣":17,"嘉義市":18,"基隆市":19,
"金門縣":20,"臺東縣":21,"澎湖縣":22, "連江縣":23}
df['縣市_label'] = df["縣市"].map(tw_region_label)

pst_label = {"軟體_工程類人員" : 1, "MIS_網管類人員":2}
df["職位類別_label"] = df['職位類別'].map(pst_label)

# pst_2_label = {"軟體設計工程師":1
# ,"Internet程式設計師":2
# ,"系統維護_操作人員":3
# ,"韌體設計工程師":4
# ,"系統分析師":5
# ,"網路管理工程師":6
# ,"其他資訊專業人員":7
# ,"MIS程式設計師":8
# ,"演算法開發工程師":9
# ,"通訊軟體工程師":10
# ,"資料庫管理人員":11
# ,"軟體專案主管":12
# ,"資訊設備管制人員":13
# ,"資訊助理人員":14
# ,"電玩程式設計師":15
# ,"MIS_網管主管":16
# ,"網路安全分析師":17
# ,"MES工程師":18
# ,"電子商務技術主管":19
# ,"BIOS工程師":20}
# df["職位_label"] = df["職位"].map(pst_2_label)

{18, 20, 22, 25, 30}


In [8]:
df["管理責任_"] = None
df["學歷要求_"] = None
df["需求人數_"] = None
df['職位_'] = None

for i in range(len(df.values)):
            # 管理責任
    manage = df.loc[i, '管理責任']
    if manage == 0 or manage == 4:
        df.loc[i, '管理責任_'] = "管理 0-4 人"
    elif manage == 8 or manage == 12:
        df.loc[i, '管理責任_'] = "管理 5-12 人"
    elif manage == 16:
        df.loc[i, '管理責任_'] = "管理 13 人以上"

    # {'高中':18,'專科':20, '大學':22, '碩士':25, '博士':30}
    edu = df.loc[i, '學歷要求']
    if edu == 18 or edu == 20 or edu == 22:
        df.loc[i, '學歷要求_'] = "高中-專科-大學"
    elif edu == 25:
        df.loc[i, '學歷要求_'] = "碩士"
    elif edu == 30:
        df.loc[i, '學歷要求_'] = "博士"

    # {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
    # 12, 13, 15, 18, 20, 21, 25, 30, 50, 51, 80, 99}
    demand = df.loc[i, '需求人數']
    if demand < 10:
        df.loc[i, '需求人數_'] = "需求 0-9 人"
    elif demand >= 10 and demand < 15:
        df.loc[i, '需求人數_'] = "需求 10-14 人"
    elif edu >= 15:
        df.loc[i, '需求人數_'] = "需求 15 人以上"

    # {'MIS程式設計師', '軟體專案主管', 'MES工程師', '資訊設備管制人員', 
    # '資料庫管理人員', '系統分析師', 'Internet程式設計師', 'BIOS工程師', 
    # '其他資訊專業人員', '演算法開發工程師', '網路安全分析師', '資訊助理人員', 
    # '通訊軟體工程師', '系統維護_操作人員', '網路管理工程師', '韌體設計工程師', 
    # 'MIS_網管主管', '電子商務技術主管', '電玩程式設計師', '軟體設計工程師'}

    grp1 = ['BIOS工程師']
    grp2 = ['電玩程式設計師','演算法開發工程師', 'Internet程式設計師', '軟體設計工程師']
    grp3 = ['系統分析師', 'MIS_網管主管', '系統維護_操作人員', '韌體設計工程師','資料庫管理人員',
    '資訊助理人員', 'MIS程式設計師', '其他資訊專業人員', '網路管理工程師', '資訊設備管制人員']
    grp4 = ['網路安全分析師', 'MES工程師', '通訊軟體工程師','軟體專案主管', '電子商務技術主管' ]

    pst = df.loc[i, '職位']
    if pst in grp2:
        df.loc[i, '職位_'] = "職位 II"
    elif pst in grp3:
        df.loc[i, '職位_'] = "職位 III"
    elif pst in grp4:
        df.loc[i, '職位_'] = "職位 IV"
    elif pst in grp1:
        df.loc[i, "職位_"] = '職位 I'
    

In [9]:
print(set(df["職位_"].values))

{'職位 III', '職位 IV', '職位 I', '職位 II'}


In [10]:
# get dummies

# df_pst = pd.get_dummies(df['職位'])
df_pst_ = pd.get_dummies(df['職位_'])
df_pst_cat = pd.get_dummies(df['職位類別'])
df_country = pd.get_dummies(df['縣市'])
df_area = pd.get_dummies(df['地區'])
df_time = pd.get_dummies(df['上班時段'])

df_edu_ = pd.get_dummies(df['學歷要求_'])
df_res_ = pd.get_dummies(df['管理責任_'])
df_dem_ = pd.get_dummies(df['需求人數_'])
df_work_ = pd.get_dummies(df['工作經歷'])
# df_pst_cat_ = pd.get_dummies(df['職位類別'])
# df_county_ = pd.get_dummies(df['縣市'])
# df_area_ = pd.get_dummies(df['地區'])
# df_time_ = pd.get_dummies(df['上班時段'])

#擅長工具
df_tools = (df.iloc[:,193:-4]).copy()
df = df.astype({'供需人數':'int64'})

In [11]:
# select analysis data
# col_list =['職位類別', '職位_','縣市', '地區', '供需人數', '需求人數_', '管理責任_',
#            '上班時段', '工作經歷', '學歷要求_', '外商', '工作待遇_min']
# dfana = df.loc[:,col_list]

# reassign analysis data
# col_list2 =['外商','供需人數', '工作待遇_min']
# dfana_final = df.loc[:, col_list2].reset_index(drop=True)

# df_test_select = pd.concat([dfana_final, df_pst_, df_pst_cat, df_country, df_area, df_time,
#                             df_edu_, df_res_, df_dem_, df_work_, df_tools], axis=1)

In [12]:
print(df_edu_.columns)
print(df_res_.columns)
print(df_dem_.columns)
print(df_work_.columns)
print(df_pst_cat.columns)
print(df_pst_.columns)
print(df_country.columns)
print(df_area.columns)
print(df_time.columns)
print(df_tools.columns)

Index(['博士', '碩士', '高中-專科-大學'], dtype='object')
Index(['管理 0-4 人', '管理 13 人以上', '管理 5-12 人'], dtype='object')
Index(['需求 0-9 人', '需求 10-14 人', '需求 15 人以上'], dtype='object')
Index(['0-1年', '1-3年', '10年以上', '3-5年', '5-10年'], dtype='object')
Index(['MIS_網管類人員', '軟體_工程類人員'], dtype='object')
Index(['職位 I', '職位 II', '職位 III', '職位 IV'], dtype='object')
Index(['南投縣', '嘉義市', '嘉義縣', '基隆市', '宜蘭縣', '屏東縣', '彰化縣', '新北市', '新竹市', '新竹縣',
       '桃園市', '澎湖縣', '臺中市', '臺北市', '臺南市', '臺東縣', '花蓮縣', '苗栗縣', '金門縣', '雲林縣',
       '高雄市'],
      dtype='object')
Index(['七堵區', '三峽區', '三民區', '三義鄉', '三芝區', '三重區', '中區', '中和區', '中埔鄉', '中壢區',
       ...
       '鹽水區', '鹿港鎮', '鹿草鄉', '麟洛鄉', '麥寮鄉', '麻豆區', '鼓山區', '龍井區', '龍潭區', '龜山區'],
      dtype='object', length=180)
Index(['日班', '輪班'], dtype='object')
Index(['3G', '3ds-Max', '3ds-Max-Design', '6-Sigma', 'A+', 'ABAQUS', 'ADA',
       'ADC', 'ADO', 'ADSL',
       ...
       '英文打字50~75', '英文打字75~100', '金旭飯店管裡系統', '金蝶', '鉅茂', '鼎基-ERP', '鼎新',
       '上班時段_label', '縣市_label', '職位

df label encoding

In [13]:
col_list = ['職位類別_label', '縣市_label', '上班時段_label', '外商', '供需人數', '工作待遇_min']
df_ = df.loc[:, col_list].reset_index(drop=True)

df_label = pd.concat([df_, df_pst_, df_area, df_edu_, df_res_, df_dem_, df_work_, df_tools], axis=1)

In [14]:
# drop predict data
df_label = df_label.drop(df_label[pd.isnull(df_label["工作待遇_min"])].index)
df_label = df_label.drop(df_label[df_label["工作待遇_min"] == 'Y'].index)

df_label = df_label.astype({"工作待遇_min":'int64'})
print(set(df_label.loc[:,'工作待遇_min']))

{64000, 33800, 85000, 42000, 41000, 27692, 30769, 32307, 35384, 48696, 36923, 38461, 40000, 1000000, 34370, 43076, 35400, 46153, 33866, 82000, 30800, 34900, 53846, 39000, 30300, 44125, 45150, 60000, 61538, 64615, 69230, 38000, 42611, 59000, 33400, 76923, 80000, 41600, 30850, 51846, 84615, 37000, 86153, 58000, 32400, 92307, 36500, 31900, 100000, 36000, 25250, 27810, 31400, 57000, 107692, 115384, 35000, 120000, 56000, 123076, 42700, 34000, 130769, 32466, 28371, 25300, 55000, 36570, 33500, 138461, 28900, 33000, 54000, 32500, 153846, 75000, 36600, 32000, 160000, 53000, 31500, 35600, 31000, 176923, 35100, 52000, 26400, 30500, 43307, 30000, 34100, 34615, 51000, 28473, 36153, 37692, 37180, 42307, 29000, 33100, 48461, 50000, 28500, 56153, 57692, 59230, 92000, 28000, 65384, 49000, 27500, 28015, 70000, 38769, 230769, 73076, 27000, 48000, 35200, 26500, 30200, 47500, 26000, 90000, 30100, 57750, 47000, 25500, 68000, 29600, 89000, 110000, 46000, 750000, 34230, 30648, 27070, 41409, 45000, 130000, 276

df get dummies

In [15]:
col_list2 = ['外商', '供需人數', '工作待遇_min']
df2_ = df.loc[:, col_list2].reset_index(drop=True)

df_dummies = pd.concat([df2_, df_pst_cat, df_pst_, df_country, df_area, df_time,
                        df_edu_, df_res_, df_dem_, df_work_, df_tools], axis=1)

In [16]:
# drop predict data
df_dummies = df_dummies.drop(df_dummies[pd.isnull(df_dummies["工作待遇_min"])].index)
df_dummies = df_dummies.drop(df_dummies[df_dummies["工作待遇_min"] == 'Y'].index)

df_dummies = df_dummies.astype({"工作待遇_min":'int64'})
print(set(df_dummies.loc[:,'工作待遇_min']))

{64000, 33800, 85000, 42000, 41000, 27692, 30769, 32307, 35384, 48696, 36923, 38461, 40000, 1000000, 34370, 43076, 35400, 46153, 33866, 82000, 30800, 34900, 53846, 39000, 30300, 44125, 45150, 60000, 61538, 64615, 69230, 38000, 42611, 59000, 33400, 76923, 80000, 41600, 30850, 51846, 84615, 37000, 86153, 58000, 32400, 92307, 36500, 31900, 100000, 36000, 25250, 27810, 31400, 57000, 107692, 115384, 35000, 120000, 56000, 123076, 42700, 34000, 130769, 32466, 28371, 25300, 55000, 36570, 33500, 138461, 28900, 33000, 54000, 32500, 153846, 75000, 36600, 32000, 160000, 53000, 31500, 35600, 31000, 176923, 35100, 52000, 26400, 30500, 43307, 30000, 34100, 34615, 51000, 28473, 36153, 37692, 37180, 42307, 29000, 33100, 48461, 50000, 28500, 56153, 57692, 59230, 92000, 28000, 65384, 49000, 27500, 28015, 70000, 38769, 230769, 73076, 27000, 48000, 35200, 26500, 30200, 47500, 26000, 90000, 30100, 57750, 47000, 25500, 68000, 29600, 89000, 110000, 46000, 750000, 34230, 30648, 27070, 41409, 45000, 130000, 276

## cluster Heatmap

In [ ]:
df_heatmap = df[['縣市','職位','管理責任','上班時段','工作經歷_數字','工作經歷','職位類別','需求人數','供需人數','外商','地區','學歷要求','工作待遇_min']]
df_heatmap = df_heatmap.drop(df[df["工作待遇_min"] == 'Y'].index)
df_heatmap = df_heatmap.drop(df[pd.isnull(df["工作待遇_min"])].index)
df_heatmap = df_heatmap.astype({"工作待遇_min":'int64'})
df_heatmap

df_heatmap_data = pd.pivot_table(df_heatmap, values='工作待遇_min',
                            index=['工作經歷'],
                            columns='縣市',fill_value=0)
df_heatmap_data.head()


In [ ]:

from matplotlib import pyplot as plt
from matplotlib.font_manager import FontProperties as font

import matplotlib
print(matplotlib.matplotlib_fname())
print(matplotlib.get_cachedir())

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Microsoft JhengHei']
# plt.rcParams['font.family']='sans-serif' 
# plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# make heatmap with Seaborn ClusterMap
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.clustermap(df_heatmap_data)
plt.savefig('heatmap_with_Seaborn_clustermap_工作經歷_類別_縣市.jpg',
            dpi=150)


In [ ]:
from interpret.glassbox import (LinearRegression,
                                RegressionTree, 
                                ExplainableBoostingRegressor)

import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.style.use('seaborn')

## sharply value 

In [ ]:
# df_heatmap.groupby("職位").size()
# df_heatmap.groupby("職位").agg(['nunique'])

df_test_select = df_test_select.reset_index(drop=True)



In [ ]:
df_test_select.columns.duplicated()

df_test_select = df_test_select.loc[:,~df_test_select.columns.duplicated()]

# df_test_select.columns.duplicated()

In [ ]:
# split data
y = df_test_select['工作待遇_min']
X = df_test_select.drop('工作待遇_min',axis=1).copy()
X.head()
y.head()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)


# check data type
print('x train data shape = ',X_train.shape)
print('y test data shape =',X_test.shape)




In [ ]:
# col_list3 =['供需人數', '管理責任', '工作經歷_數字', '學歷要求', '外商']

# X_train.columns.to_list()
# X_train = X_train[col_list3]
# X_test = X_test[col_list3]

# X_train['外商'].values



In [ ]:
print(X_train.shape)
y_train.shape


len(set(X_train.columns))


In [ ]:
# model = xgb.XGBRegressor(max_depth=4, learning_rate=0.05, n_estimators=500)
# model.fit(X_train, y_train)


# XGBoost
regressor = xgb.XGBRegressor(n_estimators=1000,reg_lambda=1,gamma=0,max_depth=5,learning_rate=0.05)

regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('XG Boosting:')
print('R-squared: {}'.format(r2))
print('MSE: {}'.format(mse))

In [ ]:
print("Best Iteration: {}".format(regressor.get_booster().best_iteration))

In [ ]:
plt.figure(figsize=(15, 6))
plt.bar(range(len(X_train.columns)),regressor.feature_importances_)
plt.xticks(range(len(X_train.columns)), X_train.columns, rotation=-45, fontsize=14)
plt.title('Feature importance', fontsize=14)
plt.show()
plt.savefig('feature_important.jpg')

In [ ]:
pd.set_option('display.max_columns', None)
feature_important = regressor.feature_importances_
feature_cols = X_train.columns
 
# Create the pandas DataFrame with column name is provided explicitly
# df = pd.DataFrame(feature_important, columns=feature_cols)
 
# print(feature_important)
# print(feature_cols)

feature = pd.DataFrame(feature_important, index=feature_cols, columns=['important'])
feature.head(20)

feature.sort_values(by=['important'], ascending=False)
feature
feature.groupby("important").size()
# feature.groupby("important").agg(['nunique'])

In [ ]:
# saving the dataframe
feature.to_csv('feaIm.csv', encoding='utf-8-sig')

In [ ]:
import shap
import tqdm as notebook_tqdm


# explainer = shap.TreeExplainer(regressor)

explainer = shap.Explainer(regressor)
shap_values = explainer(X_train)


In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
X_train.columns

In [ ]:
feature_shap = pd.DataFrame(shap_values[0].values, index = X_train.columns)
feature_shap.to_csv('feaIm_shap.csv', encoding='utf-8-sig')

In [ ]:
shap.summary_plot(shap_values, X_train)

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

## Model training

In [17]:
# 兩種版本
df_test_select = df_label.copy()
# df_test_select = df_dummies.copy()

In [18]:
# split data
y = df_test_select['工作待遇_min']
X = df_test_select.drop('工作待遇_min',axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


# check data type
print('x train data shape = ',X_train.shape)
print('y test data shape =',X_test.shape)

x train data shape =  (9069, 706)
y test data shape = (1008, 706)


In [ ]:
# Fit linear regression model
lr = LinearRegression(random_state=2022, feature_names=X.columns)
lr.fit(X_train, y_train, )
print("Training finished.")


# Evaluate linear regression model
y_pred = lr.predict(X_test)
print(f"Root Mean Squared Error: {round(mean_squared_error(y_test, y_pred)**(1/2), 2)}")
print(f"R2: {round(r2_score(y_test, y_pred), 2)}")

In [ ]:
# Fit decision tree model for regression
rt = RegressionTree(random_state=2022, max_depth=6)
rt.fit(X_train, y_train)
print("Training finished.")

# Evaluate decision tree model for regression
y_pred = rt.predict(X_test)
print(f"Root Mean Squared Error: {round(mean_squared_error(y_test, y_pred)**(1/2), 2)}")
print(f"R2: {round(r2_score(y_test, y_pred), 2)}")

In [ ]:
# Fit explainable boosting machine for regression
ebr = ExplainableBoostingRegressor(random_state=2022)
ebr.fit(X_train, y_train) 
print("Training finished.")

# Evaluate explainable boosting machine for regression
y_pred = ebr.predict(X_test)
print(f"Root Mean Squared Error: {round(mean_squared_error(y_test, y_pred)**(1/2), 2)}")
print(f"R2: {round(r2_score(y_test, y_pred), 2)}")

In [ ]:
# Lasso

model = linear_model.Lasso(alpha=1, max_iter = 100000)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('Lasso\n')
print("Mean squared error: {}".format((mean_squared_error(y_test, y_pred))))
print('R2 score: {}'.format(r2_score(y_test, y_pred)))
print('number of model coef: {}'.format(np.sum(model.coef_ != 0)))

In [ ]:
# Ridge

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

model = linear_model.Ridge(alpha=1.0, max_iter = 100000)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('Ridge\n')
print("Mean squared error: {}".format((mean_squared_error(y_test, y_pred))))
print('R2 score: {}'.format(r2_score(y_test, y_pred)))
print('number of model coef: {}'.format(np.sum(model.coef_ != 0)))

In [ ]:
# Elastic

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

model = linear_model.ElasticNet(alpha=1.0, l1_ratio=0.5, l2_ratio = 1, max_iter = 100000)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('Elastic\n')
print("Mean squared error: {}".format((mean_squared_error(y_test, y_pred))))
print('R2 score: {}'.format(r2_score(y_test, y_pred)))
print('number of model coef: {}'.format(np.sum(model.coef_ != 0)))

In [ ]:
# KNN

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


model = neighbors.KNeighborsClassifier(n_neighbors=3)   # KNN K值是3
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
con_matrix = confusion_matrix(y_test, y_pred)

print('KNN\n')
print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
print('confusion matrix: {}'.format(con_matrix))

In [ ]:
# Random Forest

os.environ["PATH"] += os.pathsep + 'C:/Users/student/Desktop/BDSE_機器學習/ands-on_part5/example/example/release/bin'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)


model = RandomForestRegressor(max_depth=3, random_state=0)
model.fit(X_train, y_train)

X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)



mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print('Random Forest:')
print('R-squared: {}'.format(r2))
print('MSE: {}'.format(mse))

In [ ]:
# Ada Boost

from sklearn.ensemble import BaggingRegressor
from sklearn import metrics

ensemble = BaggingRegressor(n_estimators=1000)

ensemble.fit(X_train, y_train)
predictions = ensemble.predict(X_test)

r2 = metrics.r2_score(y_test, predictions)
mse = metrics.mean_squared_error(y_test, predictions)

print('Gradient Boosting:')
print('R-squared: {}'.format(r2))
print('MSE: {}'.format(mse))

In [20]:
# XGBoost
regressor = xgb.XGBRegressor(n_estimators=4500,reg_lambda=10,gamma=0,max_depth=8,n_jobs = -1, random_state=0)

regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('XG Boosting:')
print('R-squared: {}'.format(r2))
print('MSE: {}'.format(mse))

XG Boosting:
R-squared: 0.8473699167242392
MSE: 45285012.666738994


In [21]:
recipe1= [2000, 3000, 4000, 5000, 6000]
recipe2= [1, 3, 5, 7, 9]
recipe3=[9,11,13,15]
results=[]
for i in recipe1:
    for j in recipe2:
        for k in recipe3:
            # XGBoost
            regressor = xgb.XGBRegressor(n_estimators=i,reg_lambda=j,gamma=0,max_depth=k,n_jobs = -1, random_state=0)
            regressor.fit(X_train, y_train)

            y_pred = regressor.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            result={
                'XG Boosting' : 'n_estimators:'+str(i)+',reg_lambda:'+str(j)+',max_depth:'+str(k) ,
                'R-squared': r2,
                'MSE:':mse
            }
            results.append(result)
            print(result)
            # print('XG Boosting:')
            # print('R-squared: {}'.format(r2))
            # print('MSE: {}'.format(mse))

{'XG Boosting': 'n_estimators:2000,reg_lambda:1,max_depth:9', 'R-squared': 0.8668644390422212, 'MSE:': 39501030.43233999}
{'XG Boosting': 'n_estimators:2000,reg_lambda:1,max_depth:11', 'R-squared': 0.8713918062001482, 'MSE:': 38157770.47536754}
{'XG Boosting': 'n_estimators:2000,reg_lambda:1,max_depth:13', 'R-squared': 0.8705913668719737, 'MSE:': 38395259.077466026}
{'XG Boosting': 'n_estimators:2000,reg_lambda:1,max_depth:15', 'R-squared': 0.8658017182573637, 'MSE:': 39816337.37033418}
{'XG Boosting': 'n_estimators:2000,reg_lambda:3,max_depth:9', 'R-squared': 0.8671770529656933, 'MSE:': 39408278.5633742}
{'XG Boosting': 'n_estimators:2000,reg_lambda:3,max_depth:11', 'R-squared': 0.8799046801539183, 'MSE:': 35632019.34850527}
{'XG Boosting': 'n_estimators:2000,reg_lambda:3,max_depth:13', 'R-squared': 0.8881778796841815, 'MSE:': 33177379.100124262}
{'XG Boosting': 'n_estimators:2000,reg_lambda:3,max_depth:15', 'R-squared': 0.8747714399576839, 'MSE:': 37155040.51391929}
{'XG Boosting': '

In [ ]:
import sklearn

print(sklearn.metrics.SCORERS.keys())

In [ ]:
from sklearn.model_selection import StratifiedKFold
kn = xgb.XGBRegressor()
params = {'n_estimators':[1000],'reg_lambda':[1],'gamma':[0],'max_depth':range(3, 10)}

scoring = ['r2','neg_mean_squared_error','neg_root_mean_squared_error']



grid_kn = GridSearchCV(estimator = kn,
                        param_grid = params,
                        cv = 5,
                        scoring = scoring,
                        refit = 'neg_root_mean_squared_error')

grid_kn.fit(X_train, y_train)
print(grid_kn.best_score_)
print(grid_kn.best_params_)
    


In [ ]:
print(grid_kn.best_score_)
print(grid_kn.best_params_)

In [ ]:
kn = AdaBoostRegressor()
# params = {'n_estimators':[1000], 'base_estimator':DecisionTreeRegressor(max_depth=range(2, 11))}
params = {'n_estimators':[1000], 'base_estimator':[2]}

list = ['r2','neg_mean_squared_error','neg_root_mean_squared_error']


for i in list:
    grid_kn = GridSearchCV(estimator = kn,
                            param_grid = params,
                            cv = 5,
                            scoring = i,
                          error_score='raise')

    grid_kn.fit(X_train, y_train)
    print(grid_kn.best_score_)
    print(grid_kn.best_params_)
    


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import seaborn as sns

print('Pandas version:', pd.__version__)
# Pandas version: 1.3.0

tips = sns.load_dataset("tips")

tips["total_bill_cut"] = pd.cut(tips["total_bill"],
                                np.arange(0, 55, 5),
                                include_lowest=True,
                                right=False)

def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    
    print(ss.chi2_contingency(confusion_matrix))


    print(chi2)
    n = confusion_matrix.sum()
    print(n)
    phi2 = chi2 / n
    print(phi2)
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

confusion_matrix = pd.crosstab(tips["day"], tips["time"])
cramers_v(confusion_matrix.values)
print(confusion_matrix)

# Out[2]: 0.9386619340722221

confusion_matrix = pd.crosstab(tips["total_bill_cut"], tips["time"])
cramers_v(confusion_matrix.values)
# Out[3]: 0.1649870749498837

tips